In [2]:
import matplotlib.font_manager as font_manager
from matplotlib.lines import Line2D
import matplotlib.ticker as mtick
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
import numpy as np
import os
#fontpath = os.path.expanduser('~/.local/share/fonts/LinLibertine_DRah.ttf')
#prop = font_manager.FontProperties(fname=fontpath)

params = {
    "axes.titlesize" : 14,
    'axes.labelsize': 12,
    'font.size': 12,
    'legend.fontsize': 12,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    #'font.family': prop.get_name(),
    'text.usetex': True
}

mpl.rcParams.update(params)

import sys
# Local Modules
sys.path.insert(0, os.path.abspath('/scratch/horta/youtube_dataset/'))
#from helpers.plot import set_size

import warnings
warnings.filterwarnings('ignore')

### Loads data and preprocess datetime fields

In [34]:
df_sb_f = pd.read_csv("./data/df_timeseries_en.tsv.gz", compression="infer", sep="\t")
df_sb_f["datetime"] = pd.to_datetime(df_sb_f["datetime"])

TypeError: read_csv() got an unexpected keyword argument 'columns'

In [35]:
# df_sb_f (18'872'498 x )
# 0: channel
# 1: category
# 2: datetime
# 3: views
# 4: delta_views
# 5: subs
# 6: delta_subs
# 7: videos
# 8: activity

,channel,category,datetime,views,delta_views,subs,delta_subs,videos,delta_videos,activity
0,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-03,2.024946e+05,0.000000,650.222222,0.000000,5,0,3
1,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-10,3.940857e+05,191591.111111,1046.000000,395.777778,6,1,1
2,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-17,8.353938e+05,441308.083333,1501.500000,455.500000,6,0,1
3,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-24,1.104577e+06,269183.250000,1750.000000,248.500000,6,0,0
4,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-31,1.284406e+06,179828.600000,2008.300000,258.300000,6,0,0
...,...,...,...,...,...,...,...,...,...,...
18872494,UC0UeVA9YHpOEr_Ng442xiRw,Gaming,2019-09-02,6.012938e+06,232418.277778,61268.611111,1305.611111,278,2,10
18872495,UC0UeVA9YHpOEr_Ng442xiRw,Gaming,2019-09-09,6.244579e+06,231640.888889,62631.666667,1363.055556,287,9,13
18872496,UC0UeVA9YHpOEr_Ng442xiRw,Gaming,2019-09-16,6.480902e+06,236322.933333,64010.000000,1378.333333,294,7,16
18872497,UC0UeVA9YHpOEr_Ng442xiRw,Gaming,2019-09-23,6.745317e+06,264415.200000,65480.000000,1470.000000,301,7,15


In [23]:
df_vd_f = pd.read_feather("./data/yt_metadata_helper.feather")
df_vd_f["dummmy"] = 1

ArrowMemoryError: malloc of size 4294967296 failed

In [21]:
# df_vd_f (72'924'794 x 8)
# 0: categories
# 1: channel_id
# 2: dislike_count
# 3: display_id
# 4: duration
# 5: like_count
# 6: upload_date
# 7: view_count

,view_count,dummmy
0,1057.0,1
1,12894.0,1
2,1800602.0,1
3,57640.0,1
4,86368.0,1
...,...,...
72924789,4409.0,1
72924790,1172.0,1
72924791,1898.0,1
72924792,726.0,1


In [ ]:
df_ch_f = pd.read_csv("./data/df_channels_en.tsv.gz", compression="infer", sep="\t")
df_ch_f["join_date"] = pd.to_datetime(df_ch_f["join_date"])

In [6]:
num_comments = pd.read_csv("./data/num_comments.tsv.gz", compression="infer", sep="\t")
num_comments_authors = pd.read_csv("./data/num_comments_authors.tsv.gz", compression="infer", sep="\t")

### 1. Video and channel statistics

In [ ]:
# gets videos_yt count
df_vd_vid_count = dict(df_vd_f.groupby("channel_id").count().display_id) 
df_ch_f["videos_yt"] = df_ch_f["channel"].apply(lambda x: df_vd_vid_count[x])

In [ ]:
import matplotlib.transforms as transforms

fig, axs = plt.subplots(5, 5, figsize=(14, 6), 
                         gridspec_kw={"wspace": 0.25, "hspace": 0.1, 
                                      "height_ratios": [0.2, 0.8, 0.65, 0.2, 0.8]})
N = len(df_vd_f)#136470
    
kwargs = {'cumulative': False}

def plot_mean(ax, vals):
    trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
    trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
    quant95 = vals.quantile(0.95)
    quant100 = vals.quantile(1)
    plt.plot([np.log10(quant95), np.log10(quant100)], [1.15, 1.15], 
             lw=1, color = "black", transform=trans, clip_on=False)
    top5p = vals[vals >= quant95].sum() / sum(vals)
    plt.text((np.log10(quant95) + np.log10(quant100)) / 2, 1.25, 
             "Top $5\% = {}\%$".format(round(top5p*100,1)), ha="center",
        transform=trans, size=10)

def plot_box(ax, vals):
    trans = transforms.blended_transform_factory(ax.transData, ax.transAxes)
    ax.boxplot(np.log10(vals), showfliers=False, vert=False)
    ax.set_yticks([])
    
# Video stuff
df_tmp = df_vd_f.sample(N)
titles = ["Views", "Likes", "Dislikes", "Duration"]
cols = ["view_count", "like_count", "dislike_count", "duration"]
xlabels = [r'$\log_{10}(views + 1)$', r'$\log_{10}(likes + 1)$',
          r'$\log_{10}(dislikes + 1)$', r'$\log_{10}(duration)$ (in seconds)']
xticks = [[0, 2, 4, 6, 8], [0, 2, 4], [0, 2, 4], [0, 1, 2, 3, 4, 5]]

for idx, title in enumerate(titles):
    ax = axs[1, idx]
    sns.distplot(np.log10(df_tmp[cols[idx]] + 1), hist_kws=kwargs, kde=False, kde_kws=kwargs, ax=ax, norm_hist=True)
    ax.set_xlabel(xlabels[idx])
    ax.set_xticks(xticks[idx])
    xtickstmp = ax.get_xticks()
    xlimtmp = ax.get_xlim()
    
    ax = axs[0, idx]
    ax.set_title(title, pad=25, size=16)
    vals = df_tmp[cols[idx]].dropna() + 1
    plot_box(ax, vals)
    plot_mean(ax, vals)
    ax.set_xticks(xtickstmp)
    ax.set_xlim(xlimtmp)
    
# Channel stuff
df_tmp = df_ch_f
titles = ["Videos per channel", "Subscribers per channel"]
cols = ["videos_yt", "subscribers_cc"]
xlabels = [r'$\log_{10}(\#videos)$', 
           r'$\log_{10}(\#subscribers)$']

for idx, title in enumerate(titles):
    
    ax = axs[4, idx]
    sns.distplot(np.log10(df_tmp[cols[idx]] + 1), hist_kws=kwargs, kde=False, kde_kws=kwargs, ax=ax, norm_hist=True)
    ax.set_xlabel(xlabels[idx])
    xtickstmp = ax.get_xticks()
    xlimtmp = ax.get_xlim()
    
    ax = axs[3, idx]
    ax.set_title(title, pad=25, size=16)
    vals = df_tmp[cols[idx]].dropna() + 1
    plot_box(ax, vals)
    plot_mean(ax, vals)
    ax.set_xticks(xtickstmp)
    ax.set_xlim(xlimtmp)

N = len(df_sb_f)#136470

# Time series stuff
df_tmp = df_sb_f.sample(N)
titles = ["Weekly views gain", "Weekly subscriber gain"]
cols = ["delta_views", "delta_subs"]
xlabels = [r'$\log_{10}(\Delta views + 1)$', 
           r'$\log_{10}(\Delta subscribers + 1)$']

for idx, title in enumerate(titles):
    ax = axs[4, idx+2]
    sns.distplot(np.log10(df_tmp[cols[idx]] + 1), hist_kws=kwargs, 
                 kde=False, kde_kws=kwargs, ax=ax, norm_hist=True)
    ax.set_xlabel(xlabels[idx])
    xtickstmp = ax.get_xticks()
    xlimtmp = ax.get_xlim()
    
    ax = axs[3, idx+2]
    ax.set_title(title, pad=25, size=16)
    vals = df_tmp[cols[idx]].dropna() + 1
    plot_box(ax, vals)
    plot_mean(ax, vals)
    ax.set_xticks(xtickstmp)
    ax.set_xlim(xlimtmp)
    
for ax in axs[2, :]:
    ax.set_visible(False)


for ax in axs[0:2, 4]:
    ax.set_xticks([0, 2, 4, 6])
    ax.set_xlim([0, 7])
    
for ax in axs[3:, 4]:
    ax.set_xticks([0, 2, 4, 6])
    ax.set_xlim([0, 7])
    
NC = 1000000000
    
sns.distplot(np.log10(num_comments.num_comms.head(NC) + 1), kde=False, 
             kde_kws=kwargs, ax=axs[1, 4], norm_hist=True)
sns.distplot(np.log10(num_comments_authors.video_id.head(NC) + 1), kde=False, 
             kde_kws=kwargs,  ax=axs[4, 4], norm_hist=True)

# # axs[0, 0].set_title("Commments per Video", pad=25, size=16)
# # axs[0, 1].set_title("Commments per Author", pad=25, size=16)

ax = axs[0, 4]
ax.set_title("Commments per Video", pad=25, size=16)
vals = num_comments.num_comms.head(NC) + 1
plot_box(ax, vals)
plot_mean(ax, vals)
ax.set_xticks([])


ax = axs[3, 4]
ax.set_title("Commments per User", pad=25, size=16)
vals = num_comments_authors.video_id.head(NC) + 1
plot_box(ax, vals)
plot_mean(ax, vals)
ax.set_xticks([])

axs[1, 4].set_yscale("log")
axs[4, 4].set_yscale("log")


axs[1, 4].set_xlabel(r'$\log_{10}(\#comments + 1)$')
axs[4, 4].set_xlabel(r'$\log_{10}(\#comments + 1)$')

set_size(fig, size=(14, 6))
fig.savefig("./images/video_metadata.pdf", bbox_inches="tight")

### 2. Time series examples

In [ ]:
import matplotlib.dates as mdates
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')
fig, axs = plt.subplots(1, 3, figsize=(7, 2.5), sharey=True, sharex=True, gridspec_kw={"wspace": 0.05,})
np.random.seed(5)
s = df_sb_f.sample(3).channel.values
for chan, ax in zip(s, axs):
    vals = df_sb_f.loc[df_sb_f.channel == chan]
    vals.datetime = pd.to_datetime(vals.datetime)
    ax.plot(vals.datetime, vals.views, label="Cum. Views", color="#1b9e77")
    ax.plot(vals.datetime, vals.subs, label="Cum. Subscribers", color="#7570b3", ls="--")
    ax.plot(vals.datetime, vals.videos, label="Cum. Videos", color="#e6ab02", ls="-.")
    ax.set_yscale('log')
    ax.set_xlabel("Date")
    ax.xaxis.grid(color="#CCCCCC", ls=":")
    ax.yaxis.grid(color="#CCCCCC", ls=":")

axs[1].legend(bbox_to_anchor=(0.5, 1.3), ncol=3, loc='upper center')
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)

set_size(fig, size=(7, 3))
fig.savefig("./images/example_ts.pdf", bbox_inches="tight")

### 3. Time series completeness

In [ ]:
df_sb_sum = df_sb_f.groupby(pd.Grouper(key="datetime", freq="M")).agg({"channel": pd.Series.nunique})
df_sb_sum = df_sb_sum.head(-1)

In [ ]:
df_ch_sum = df_ch_f.groupby(pd.Grouper(key="join_date", freq="M")).channel.count()
df_ch_sum = df_ch_sum.cumsum()
df_ch_sum = df_ch_sum[df_sb_sum.index]

In [ ]:
fig, ax = plt.subplots(1, figsize=(7,3), sharey=True, sharex=True,
                       gridspec_kw={"wspace": 0.05})
ax.plot(df_sb_sum.channel /df_ch_sum, color="#1b9e77")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)
ax.xaxis.grid(color="#CCCCCC", ls=":")
ax.yaxis.grid(color="#CCCCCC", ls=":")
ax.set_xlabel("Date")
ax.set_ylabel("\% of channels with timeseries")
set_size(fig, size=(7, 3))
fig.savefig("./images/repr_ts.pdf", bbox_inches="tight")

### 3. Creation date for channels and videos

In [ ]:
df_ch_count = df_ch_f.groupby(pd.Grouper(key="join_date", freq="M")).count().videos_cc
df_vc_count = df_vd_f.groupby(pd.Grouper(key="upload_date", freq="M")).count().channel_id

In [ ]:
years = mdates.YearLocator(4)   # every 2 years
months = mdates.YearLocator(2)  # every year
fig, axs = plt.subplots(1, 2, figsize=(7,3), sharex=True, gridspec_kw={"wspace": 0.20})

ax = axs[0]
ax.plot(df_ch_count.cumsum(),  color="#1b9e77", label="\% channels created")
ax.plot(df_vc_count.cumsum(),  color="#7570b3", ls="--", label="\% videos uploaded")
ax.set_yscale("log")
ax.set_title("Cumulative count")
ax = axs[1]
ax.plot(df_ch_count.cumsum()/136470,  color="#1b9e77", label="\% channels created")
ax.plot(df_vc_count.cumsum()/72924794,  color="#7570b3", ls="--", label="\% videos uploaded")
ax.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
ax.set_title("Cumulative percentage")

for ax in axs:
    ax.xaxis.set_major_locator(years)
    ax.xaxis.set_major_formatter(years_fmt)
    ax.xaxis.set_minor_locator(months)
    ax.xaxis.grid(color="#CCCCCC", ls=":")
    ax.yaxis.grid(color="#CCCCCC", ls=":")
    ax.set_xlabel("Date")

axs[0].legend(bbox_to_anchor=(1.15, 1.5), ncol=3, loc='upper center')
set_size(fig, size=(7, 3))
fig.savefig("./images/overall_ch.pdf", bbox_inches="tight")

### 4. Ranking completeness

In [ ]:
import scipy.stats as stats
fig, axs = plt.subplots(1, 2, figsize=(14, 3), sharex=True, gridspec_kw={"wspace": 0.30})
 

ax = axs[0]

ranks = df_ch_f["subscriber_rank_sb"].dropna().sort_values().values
quant = np.array(list(range(1,len(ranks)+ 1)))
p = quant / ranks

ax.plot(ranks, p)#, s=10, alpha=1)
ax.set_xscale("log")
ax.set_ylabel("Percent in the sample")
ax.set_xlabel("Subscriber rank")
ax.set_title("Cumulative percentage of\nchannels in sample")
ax.xaxis.grid(color="#CCCCCC", ls=":")
ax.yaxis.grid(color="#CCCCCC", ls=":")
ax.set_xticks([10**1, 10**2, 10**3, 10**4, 10**5, 10**6])

ax = axs[1]
sns.histplot(
    df_ch_f, x="subscriber_rank_sb", y="subscribers_cc",
    bins=30, discrete=(False, False), log_scale=(True, True),
    cbar=True, cbar_kws=dict(shrink=.75),
)
ax.set_yscale("log")
ax.set_xscale("log")
ax.set_xlabel("Subscriber rank")
ax.set_ylabel("Number of subscribers ($\log$)")
ax.xaxis.grid(color="#CCCCCC", ls=":")
ax.yaxis.grid(color="#CCCCCC", ls=":")
ax.set_title(r'Subscriber rank vs. $\log(\#subscribers)$' + "\n" + r'($\rho = -0.994^{***}$)')
print(stats.spearmanr(df_ch_f[~df_ch_f["subscriber_rank_sb"].isna()]["subscriber_rank_sb"], 
                df_ch_f[~df_ch_f["subscriber_rank_sb"].isna()]["subscribers_cc"]))
ax.set_yticks([10**4, 10**5, 10**6, 10**7, 10**8])
ax.set_xticks([10**1, 10**2, 10**3, 10**4, 10**5, 10**6])

set_size(fig, size=(7, 3))
fig.savefig("./images/ranking.pdf", bbox_inches="tight")

### Table 1: Views, Videos, Likes, Duration

In [ ]:
df_vd_cat_count = df_vd_f.groupby("categories").sum()
df_vd_cat_count = df_vd_cat_count.drop(["", "Movies", "Shows"])
df_vd_cat_count[["dislike_count", "like_count"]] = df_vd_cat_count[["dislike_count", "like_count"]] / 1000000
df_vd_cat_count[["dummmy"]] = df_vd_cat_count[["dummmy"]] / 1000000
df_vd_cat_count[["view_count"]] = df_vd_cat_count[["view_count"]] / 1000000000
df_vd_cat_count[["duration"]] = df_vd_cat_count[["duration"]] / 3.154e+7
tmp = df_vd_cat_count.rename({"dislike_count": "# Dislikes (M)",
    "duration": "Duration (Years)",
    "like_count": "# Likes (M)",
    "view_count": "# Views (B)",
    "dummmy": "# Videos (M)"
    }, axis=1)[["# Views (B)", "# Videos (M)", "# Likes (M)", "Duration (Years)"]]
tmp = tmp.append(tmp.sum(axis=0).rename("Total"))
print(round(tmp,1).to_latex())

### Extra 1: Missing data on time-series data!

In [ ]:
sns.distplot(np.log10(df_sb_f[(df_sb_f.delta_subs == 0)].subs))
np.mean((df_sb_f[(df_sb_f.delta_subs == 0)].subs > 9900) & (df_sb_f[(df_sb_f.delta_subs == 0)].subs > 10100))

### Extra 2: Completeness of # videos

In [ ]:
import scipy.stats as stats
fig, ax = plt.subplots(1, 1, figsize=(7, 3), sharex=False, gridspec_kw={"wspace": 0.30})
x = abs(df_ch_f["videos_yt"] - df_ch_f["videos_cc"]) / pd.concat([df_ch_f["videos_yt"], 
                                                                  df_ch_f["videos_cc"]], axis=1).max(axis=1)


x = sorted(x)
p = 1. * np.arange(len(x))/(len(x) - 1)
p95 = abs(p - 0.95).argmin()
ax.plot(x, p, color="#e6ab02")
ax.scatter([x[p95]], [p[p95]], zorder=10, color="black", marker="X")
ax.text(x[p95] + 0.025, p[p95] -0.0825, 
        "For 95\% of all \n channels the relative  \n difference is $\leq$ 20\%", va="top")
ax.xaxis.grid(color="#CCCCCC", ls=":")
ax.yaxis.grid(color="#CCCCCC", ls=":")
ax.set_ylabel("Cumulative \% of channels")
ax.set_xlabel(r'Relative difference between \#videos' + '\n(channelcrawler.com vs. YouTube)')
ax.set_title("Completeness of videos per channel")